In [682]:
import random
import pandas as pd
import numpy as np

In [683]:
# inputs
num_tipos_pacientes = 10
num_recursos = 12
num_areas_pop = 25
num_hospitais = 10
size_square = 100

horizonte_planejado = 8

# valores maximos
estadia_max = 5 #round(horizonte_planejado/2)
qtd_pacientes_por_tipo_area_max = 400
qtd_recursos_por_hospital_max = 180
qtd_recursos_por_hospital_min = 170
distancia_max = 50
data_saida_pac_init_max = estadia_max - 1
qtd_pacientes_liberados_init_max = qtd_recursos_por_hospital_max/2


In [684]:
# tipo de paciente vs recursos usados

tipo_paciente = ["TP" + str(i) for i in range(1, num_tipos_pacientes + 1)]
recursos =  ["R" + str(i) for i in range(1, num_recursos + 1)]
usa_recurso = np.random.choice(['0', '1'], size=(num_tipos_pacientes*num_recursos))

tipo_paciente_df_in = np.tile(tipo_paciente, num_recursos)
tipo_paciente_df = sorted(tipo_paciente_df_in)
recursos_df = np.tile(recursos, num_tipos_pacientes)

#print(tipo_paciente)
#print(recursos)
#print(usa_recurso)

paciente_vs_recurso = pd.DataFrame({'Tipo_de_paciente': tipo_paciente_df, 'Recurso': recursos_df, 'Usa_recurso': usa_recurso})
paciente_vs_recurso['Usa_recurso'] = pd.to_numeric(paciente_vs_recurso['Usa_recurso'])
  
#display(paciente_vs_recurso.sort_values(by=['Tipo_de_paciente', 'Recurso']))

#fazer check se para cada tipo de paciente tem pelo menos 1 recurso sendo alocado
for tipo_paciente_in in paciente_vs_recurso['Tipo_de_paciente'].unique():
  filtro_tipo = paciente_vs_recurso['Tipo_de_paciente'] == tipo_paciente_in
 
  soma_recursos = paciente_vs_recurso.loc[filtro_tipo, 'Usa_recurso'].sum()
  
  if soma_recursos == 0:

      filtro_recurso_1 = paciente_vs_recurso['Recurso'] == 'R1'
      
      paciente_vs_recurso.loc[filtro_tipo & filtro_recurso_1, 'Usa_recurso'] = 1

paciente_vs_recurso.sort_values(by=['Tipo_de_paciente', 'Recurso'])

,Tipo_de_paciente,Recurso,Usa_recurso
0,TP1,R1,1
9,TP1,R10,1
10,TP1,R11,0
11,TP1,R12,0
1,TP1,R2,0
...,...,...,...
112,TP9,R5,0
113,TP9,R6,0
114,TP9,R7,0
115,TP9,R8,1


In [685]:
# tipo de paciente vs tempo de estadia
amostras_normais = np.random.normal(loc=estadia_max / 2, scale=estadia_max / 5, size=num_tipos_pacientes)
amostras_limite = np.clip(np.round(amostras_normais), 1, estadia_max)
amostras_limite = [max(elem, 0)  for elem in amostras_limite]
estadia = amostras_limite #np.random.randint(1, estadia_max, size=num_tipos_pacientes)

#print(tipo_paciente)
#print(estadia)

tipo_paciente_estadia_df = pd.DataFrame({'Tipo_de_paciente': tipo_paciente, 'Estadia': estadia})
display(tipo_paciente_estadia_df)

,Tipo_de_paciente,Estadia
0,TP1,2.0
1,TP2,4.0
2,TP3,2.0
3,TP4,2.0
4,TP5,4.0
5,TP6,4.0
6,TP7,4.0
7,TP8,3.0
8,TP9,2.0
9,TP10,3.0


In [686]:
# tipo de paciente vs área populacional vs dia de entrada vs qtd pacientes
area_pop = ["AP" + str(i) for i in range(1, num_areas_pop + 1)]
dias_planejados = [int(i) for i in range(1, horizonte_planejado + 1)] 
tipo_paciente = ["TP" + str(i) for i in range(1, num_tipos_pacientes + 1)]


qtd_pacientes_normais = np.random.normal(
    loc=qtd_pacientes_por_tipo_area_max / 2, 
    scale=qtd_pacientes_por_tipo_area_max/5, 
    size=num_tipos_pacientes * num_areas_pop * horizonte_planejado
)
qtd_pacientes_limite = np.clip(np.round(qtd_pacientes_normais), 0, qtd_pacientes_por_tipo_area_max)
qtd_pacientes_limite = [max(elem, 0) for elem in qtd_pacientes_limite]

qtd_pacientes = qtd_pacientes_limite #np.random.randint(0, qtd_pacientes_por_tipo_area_max, size=num_tipos_pacientes*num_areas_pop*horizonte_planejado)

tipo_paciente_df = np.tile(tipo_paciente, num_areas_pop*horizonte_planejado)

area_pop_df_in = np.tile(area_pop, num_tipos_pacientes*horizonte_planejado)
area_pop_df = sorted(area_pop_df_in)

dias_planejados_df_in1 = np.tile(dias_planejados, num_areas_pop)

dias_planejados_df_in2 = sorted(dias_planejados_df_in1)

dias_planejados_df = np.tile(dias_planejados_df_in2, num_tipos_pacientes)

area_pop_qtd_pac_df = pd.DataFrame({'Area_pop': area_pop_df, 'Tipo_de_paciente': tipo_paciente_df, 'Qtd_pacientes': qtd_pacientes, 'Dia': dias_planejados_df})
display(area_pop_qtd_pac_df)

,Area_pop,Tipo_de_paciente,Qtd_pacientes,Dia
0,AP1,TP1,78.0,1
1,AP1,TP2,275.0,1
2,AP1,TP3,222.0,1
3,AP1,TP4,105.0,1
4,AP1,TP5,107.0,1
...,...,...,...,...
1995,AP9,TP6,134.0,8
1996,AP9,TP7,180.0,8
1997,AP9,TP8,183.0,8
1998,AP9,TP9,70.0,8


In [687]:
# hospitais vs recursos vs quantidade de recursos
hospitais = ["H" + str(i) for i in range(1, num_hospitais + 1)]

media = (qtd_recursos_por_hospital_min + qtd_recursos_por_hospital_max) / 2
desvio_padrao = (qtd_recursos_por_hospital_max - qtd_recursos_por_hospital_min) / 6
qtd_recursos_normais = np.random.normal(
    loc=media, 
    scale=desvio_padrao, 
    size=num_recursos * num_hospitais
)
qtd_recursos_limite = np.clip(np.round(qtd_recursos_normais), qtd_recursos_por_hospital_min, qtd_recursos_por_hospital_max)
qtd_recursos_limite = [max(elem,0) for elem in qtd_recursos_limite]

qtd_recursos = qtd_recursos_limite # np.random.randint(qtd_recursos_por_hospital_min, qtd_recursos_por_hospital_max, size=num_recursos*num_hospitais)

hospitais_df = np.tile(hospitais, num_recursos)
recursos_df_in = np.tile(recursos, num_hospitais)
recursos_df = sorted(recursos_df_in)
#print(hospitais_df)
#print(qtd_recursos)

hospitais_vs_rec_qtd_df = pd.DataFrame ({'Hospital': hospitais_df, 'Recurso': recursos_df, 'Qtd_recurso': qtd_recursos})
display(hospitais_vs_rec_qtd_df.sort_values(by=['Hospital','Recurso']))

,Hospital,Recurso,Qtd_recurso
0,H1,R1,172.0
10,H1,R10,175.0
20,H1,R11,175.0
30,H1,R12,175.0
40,H1,R2,175.0
...,...,...,...
78,H9,R5,174.0
88,H9,R6,174.0
98,H9,R7,172.0
108,H9,R8,175.0


In [688]:
# Função para calcular a distância euclidiana
def calcular_distancia(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# Função principal
def calcular_distancias(hospitais, area_pop):
    # Dicionários para armazenar as localizações dos hospitais e áreas populacionais
    localizacao_hospitais = {}
    localizacao_area_pop = {}
    
    # Atribuir localizações aleatórias no plano cartesiano (x, y entre 0 e 100)
    for hospital in hospitais:
        localizacao_hospitais[hospital] = (random.randint(0, size_square), random.randint(0, size_square))
        
    for area in area_pop:
        localizacao_area_pop[area] = (random.randint(0, size_square), random.randint(0, size_square))
    
    # Criar lista para armazenar as distâncias
    distancias = []
    
    # Calcular as distâncias euclidianas entre cada hospital e cada área populacional
    for hospital in hospitais:
        for area in area_pop:
            dist = calcular_distancia(localizacao_hospitais[hospital], localizacao_area_pop[area])
            distancias.append([hospital, area, round(dist, 2)])  # Arredonda a distância para 2 casas decimais
    
    # Criar DataFrame
    df = pd.DataFrame(distancias, columns=["Hospital", "Area_pop", "Distancia"])
    
    return df


In [689]:
# hospitais vs area pop vs distância (gerando aleatoriamente pontos num plano cartesiano)
hospitais_vs_area_pop_dist_df = calcular_distancias(hospitais, area_pop)
hospitais_vs_area_pop_dist_df

,Hospital,Area_pop,Distancia
0,H1,AP1,4.47
1,H1,AP2,52.15
2,H1,AP3,19.42
3,H1,AP4,48.30
4,H1,AP5,58.18
...,...,...,...
245,H10,AP21,60.42
246,H10,AP22,53.04
247,H10,AP23,14.87
248,H10,AP24,57.25


In [690]:
# # hospitais vs area pop vs distância
# distancia = np.random.randint(1, distancia_max, size=num_hospitais*num_areas_pop)

# hospitais_dist_df = np.tile(hospitais, num_areas_pop)
# area_pop_dist_df_in = np.tile(area_pop, num_hospitais)
# area_pop_dist_df = sorted(area_pop_dist_df_in)

# hospitais_vs_area_pop_dist_df = pd.DataFrame ({'Hospital': hospitais_dist_df, 'Area_pop': area_pop_dist_df, 'Distancia': distancia})
# display(hospitais_vs_area_pop_dist_df.sort_values(by=['Hospital','Area_pop']))

In [691]:
area_pop

['AP1',
 'AP2',
 'AP3',
 'AP4',
 'AP5',
 'AP6',
 'AP7',
 'AP8',
 'AP9',
 'AP10',
 'AP11',
 'AP12',
 'AP13',
 'AP14',
 'AP15',
 'AP16',
 'AP17',
 'AP18',
 'AP19',
 'AP20',
 'AP21',
 'AP22',
 'AP23',
 'AP24',
 'AP25']

In [692]:
# hospitais vs area pop vs distância
num_areas_pop

25

In [693]:
# condição inicial: hospitais vs tipo paciente vs qtd vs data de saída
data_saida = np.random.randint(1, data_saida_pac_init_max, size=num_hospitais*num_tipos_pacientes)
qtd_pacientes_liberados_init = np.random.randint (0, qtd_pacientes_liberados_init_max, size = num_hospitais*num_tipos_pacientes)

print(hospitais)
print(tipo_paciente)
print(qtd_pacientes_liberados_init)
print(data_saida)

hospitais_lib_df_in = np.tile(hospitais, num_tipos_pacientes)
hospitais_lib_df = sorted(hospitais_lib_df_in)
tipo_paciente_lib_df = np.tile(tipo_paciente, num_hospitais)

cond_init_df = pd.DataFrame({'Hospital': hospitais_lib_df, 'Tipo_de_paciente': tipo_paciente_lib_df, 'Qtd_pacientes_liberados': qtd_pacientes_liberados_init, 'Dia': data_saida})
display(cond_init_df)

['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10']
['TP1', 'TP2', 'TP3', 'TP4', 'TP5', 'TP6', 'TP7', 'TP8', 'TP9', 'TP10']
[89 62  2 40 39 19 57 13 41 80 73 34 83 76  1 63 59 24 28 53 23 55  2 24
 73 14 34 21 19 53 27 89 85 52 54  7 59 19 46 63 72 21 75 17 65  4 49 86
 21 55 61 41 42 71 74 74 21 68 45 83  2 41  5 81 34 70  9 81 72 57 24 47
 83 20 10 44 71 81 22  1  3 33 71 56 69 43 48 32 88 30  3 29 12 31 71 20
 37 71 42 45]
[3 2 2 2 1 1 1 1 1 2 2 2 2 1 3 2 1 3 3 3 3 2 3 3 2 1 1 3 2 2 2 2 2 1 2 1 3
 2 2 1 1 2 3 2 1 1 3 3 1 2 3 2 3 1 2 3 3 2 1 1 1 3 1 2 3 1 1 2 1 3 2 3 3 3
 3 2 2 2 2 2 3 3 3 1 1 1 1 3 1 2 1 1 3 2 1 3 3 3 1 3]


,Hospital,Tipo_de_paciente,Qtd_pacientes_liberados,Dia
0,H1,TP1,89,3
1,H1,TP2,62,2
2,H1,TP3,2,2
3,H1,TP4,40,2
4,H1,TP5,39,1
...,...,...,...,...
95,H9,TP6,20,3
96,H9,TP7,37,3
97,H9,TP8,71,3
98,H9,TP9,42,1


In [694]:
paciente_vs_recurso.to_csv('../data/paciente_vs_recurso.csv', index=False)
tipo_paciente_estadia_df.to_csv('../data/tipo_paciente_estadia.csv', index=False)
area_pop_qtd_pac_df.to_csv('../data/area_pop_qtd_pacientes.csv', index=False)
hospitais_vs_rec_qtd_df.to_csv('../data/hospitais_vs_recursos_qtd.csv', index=False)
hospitais_vs_area_pop_dist_df.to_csv('../data/hospitais_vs_area_pop_dist.csv', index=False)
cond_init_df.to_csv('../data/cond_init.csv', index=False)